In [1]:
import numpy as np
import dask.dataframe as dd
import pandas as pd
from mlt.default import selector as s
import cv2, os
import matplotlib.pyplot as plt
%matplotlib inline

# Configs

In [2]:
ATTR_DATA_FILE = "/home/jovyan/vmldata/tmp_shrek/20190104_raw_data/raw_attribute_data_shrek_cleaned.h5"
IMAGE_DATA_FILE = "/home/jovyan/vmldata/tmp_shrek/20190104_raw_data/raw_image_data_patrick.h5"



# Please choose limited CATEGORIES. This will avoid Memory issues.
SELECTED_CATEGORIES = ['BOOT', 'BOOTS', "PUMP", "PUMPS", "FLAT", "FLATS", 
                       "HANDBAG", "HANDBAGS", 
                       "WATCH", "WATCHES",
                       "PANTS", "JEANS", "DRESS",
                       "TOP", "TOPS", "SWEATER", "SWEATERS", "COAT", "COATS"]

SELECTED_CATEGORIES = ['PUMP', 'PUMPS']

# optional config. Make it empty list if not used.
SELECTED_ATTRIBUTES = ['Handbag Style', 'Handbag Size', 'Handbag Occasion', 'Handbag Material']
SELECTED_ATTRIBUTES = []

# CHANGE THIS
SAVE_OUTPUT_AS = "/home/jovyan/vmldata/tmp_shrek/PUMP_VALIDATION_DATA.h5"

# Configs Validation

In [3]:
if len(SELECTED_CATEGORIES) == 0:
    print("ERROR: Please limit your SELECTED_CATEGORIES. Otherwise it takes too long. May also run out of memory.")
    for _ in range(10):
        print("STOP")

# Load IDM raw files

In [4]:
# load attr file
raw_attr = pd.read_hdf(ATTR_DATA_FILE)
print("count raw_attr", len(raw_attr))
raw_attr.head()

count raw_attr 24503412


,PROD_ID,ATTR_NAME,ATTR_VAL
0,9,NRF_ID,143
1,9,NRF_ID,46
2,9,NRF_ID,640
3,9,NRF_ID,650
4,45,Age Group,Adult


In [5]:
unique_cats = raw_attr[raw_attr['ATTR_NAME'] == 'Category']['ATTR_VAL']
unique_cats = list(set(unique_cats))
unique_cats

['COOKWARE_SETS',
 'SWIMSUIT',
 'SUNCARE',
 'WATCH',
 'BUSINESS_AND_LAPTOP',
 'EAU_DE_PARFUM',
 'MIXERS',
 'CLEANING_ORGANIZING',
 'SLIPPER',
 'FLATWARE',
 'FRAMES',
 'PILLOWS',
 'BATH',
 'GARTER_BELT',
 'SPECIALTY_COOKWARE',
 'TABLE_LINENS',
 'FRAGRANCE',
 'WRAP',
 'KITCHEN_LINEN',
 'SHORT',
 'BLENDERS',
 'ELECTRICS',
 'FLAT',
 'HANDBAG',
 'ELECTRONICS',
 'LAMP',
 'SAUTÉ_PAN',
 'GLOVES',
 'TIES',
 'TOPS',
 'DECORATIVE_PILLOW',
 'PILLOWCASES',
 'SHIRT',
 'MAKEUP',
 'CONCEALER',
 'LINGERIE_SETS',
 'CLEANING/ORGANIZING',
 'RICE_COOKERS',
 'COMFORTER',
 'WALLET',
 'CHARMS',
 'SKINCARE_SET',
 'SWEATER',
 'HANDBAGS',
 'BOOTS',
 'BASE',
 'VIRTUAL_GIFT_CARD',
 'CASUAL_CHINA',
 'GARMENT_BAG',
 'BARWARE',
 'SOCK',
 'ORNAMENT',
 'NAILS',
 'SLOW_COOKER',
 'MATTRESS_PAD',
 'CURTAINS_&_DRAPES',
 'JEWELRY',
 'TREES_AND_WREATHS',
 'CHINA',
 'SERVEWARE',
 'ROBES',
 'DUVET',
 'GLASSWARE',
 'SETS',
 'BATH_ACCESSORY',
 'ESPRESSO_MAKER',
 'ART',
 'GRILLS',
 'SLIPPERS',
 'EYE_MAKEUP',
 'FINE_CHINA',
 'THON

In [6]:
# convert Top's -> TOPS 
# basically remove all single quotes and brackets so df_query will work without issues later
# 
# this step takes a long time. So I have saved its output as raw_attribute_data_shrek_cleaned.h5
def clean_category_value(row):
    name = row['ATTR_NAME']
    val = row['ATTR_VAL']
    if name == 'Category':
        val = val.upper()
        val = val.replace(" ", "_")
        val = val.replace("'", "")
    return val 

#raw_attr['ATTR_VAL'] = raw_attr.apply(clean_category_value, axis=1)
#raw_attr.head()

In [7]:
# load image file
raw_image = pd.read_hdf(IMAGE_DATA_FILE)
raw_image = raw_image[raw_image['COLORWAY_IMAGE_ROLE_TYPE'] == 'CPRI']
raw_image = raw_image[['PRODUCT_ID', 'IMAGE_ID']]

print("count raw_image", len(raw_image))

def make_image_filename(row):
    pid = str(row['PRODUCT_ID'])
    return pid[0] + '/' + pid[1] + '/' + pid + '_' + str(row['IMAGE_ID']) + '.jpg'

raw_image['IMAGE_FILE'] = raw_image.apply(make_image_filename, axis=1)

raw_image.head()

count raw_image 705487


,PRODUCT_ID,IMAGE_ID,IMAGE_FILE
0,5923807,9359780,5/9/5923807_9359780.jpg
7,5303346,9014538,5/3/5303346_9014538.jpg
8,7118640,10436245,7/1/7118640_10436245.jpg
9,8220620,10436245,8/2/8220620_10436245.jpg
10,5303354,9014637,5/3/5303354_9014637.jpg


# Filter Attributes df for Selected Categories

In [8]:
shoes_query_string = 'ATTR_NAME == "Category" and (ATTR_VAL == "SHOE" or ATTR_VAL == "SHOES")'
print("\n", "query_string:", shoes_query_string)
shoes_cat_df = raw_attr.query(shoes_query_string)
shoes_cat_pids = list(set(shoes_cat_df['PROD_ID']))
shoes_cat_df = raw_attr.loc[raw_attr['PROD_ID'].isin(shoes_cat_pids)]
print("\n", "count shoes_cat_pids", len(shoes_cat_pids))

boots_query_string = 'PROD_ID == @shoes_cat_pids and ATTR_NAME == "Shoe Type" and (ATTR_VAL == "Pump" or ATTR_VAL == "Pumps")'
print("\n", "query_string:", boots_query_string)
cat_df = raw_attr.query(boots_query_string)
cat_pids = list(set(cat_df['PROD_ID']))
cat_df = raw_attr.loc[raw_attr['PROD_ID'].isin(cat_pids)]
print("\n", "count cat_pids", len(cat_pids))


print("\n", "df shape:", cat_df.shape)

cat_df.head()


 query_string: ATTR_NAME == "Category" and (ATTR_VAL == "SHOE" or ATTR_VAL == "SHOES")

 count shoes_cat_pids 55377

 query_string: PROD_ID == @shoes_cat_pids and ATTR_NAME == "Shoe Type" and (ATTR_VAL == "Pump" or ATTR_VAL == "Pumps")

 count cat_pids 2760

 df shape: (42928, 3)


,PROD_ID,ATTR_NAME,ATTR_VAL
209043,201394,Age Group,Adult
209044,201394,Brand,GUESS
209045,201394,Category,SHOE
209046,201394,Category of Business,Women's
209047,201394,Category of Business,Women's Shoes


In [9]:
raw_attr[raw_attr['PROD_ID'] == 201394]

,PROD_ID,ATTR_NAME,ATTR_VAL
209043,201394,Age Group,Adult
209044,201394,Brand,GUESS
209045,201394,Category,SHOE
209046,201394,Category of Business,Women's
209047,201394,Category of Business,Women's Shoes
209048,201394,Color Family,Black
209049,201394,Gender,Women
209050,201394,More Colors Available,true
209051,201394,NRF_ID,1
209052,201394,Pump Type,D'Orsay


In [10]:
image_pids = list(set(raw_image['PRODUCT_ID']))

print("cat_pids", len(cat_pids))
print("image_pids", len(image_pids))


inter = list(set(cat_pids) & set(image_pids))
print("inter", len(inter))


cat_pids 2760
image_pids 417001
inter 1225


# Pivot cat_df

In [11]:
def dedup_rows(x):
    if len(x) > 1:
        o = set([i for i in x])
        return ', '.join(o)
    else:
        return x
    
aggregations = {
    'ATTR_VAL': dedup_rows
}
cat_df_pivoted = cat_df.groupby(['PROD_ID', 'ATTR_NAME']).agg(aggregations).reset_index()
cat_df_pivoted = cat_df_pivoted.pivot(index='PROD_ID', columns='ATTR_NAME', values='ATTR_VAL')
print("\n", "shape cat_df_pivoted", cat_df_pivoted.shape)
cat_df_pivoted.head()


 shape cat_df_pivoted (2760, 35)


ATTR_NAME,Age Group,Age Range (Toys),Apparel Occasion,Boot Height,Boot Type,Brand,Category,Category of Business,Cold Weather & Rain,Color Family,...,Sandal Type,Shoe Style,Shoe Trend,Shoe Type,Size Range,Special Size,Sub Brand,Toe Type,Trends,Weather
PROD_ID,,,,,,,,,,,,,,,,,,,,,
201394,Adult,NaN,NaN,NaN,NaN,GUESS,SHOE,"Women's Shoes, Women's",NaN,Black,...,NaN,NaN,NaN,Pumps,NaN,Regular,NaN,Pointy,NaN,NaN
302095,Adult,NaN,NaN,NaN,NaN,Anne Klein,SHOE,"Women's Shoes, Women's",NaN,NaN,...,NaN,NaN,5,Pumps,NaN,NaN,NaN,Square,NaN,NaN
323268,Adult,NaN,NaN,NaN,NaN,Michael Kors,SHOE,"Women's Shoes, Women's",NaN,"Red, Black",...,NaN,Loafers,NaN,Pumps,NaN,NaN,MICHAEL Michael Kors,Round,NaN,NaN
331200,Adult,NaN,NaN,NaN,NaN,Michael Kors,SHOE,"Women's Shoes, Women's",NaN,NaN,...,NaN,NaN,NaN,Pumps,NaN,NaN,MICHAEL Michael Kors,NaN,NaN,NaN
373643,Adult,NaN,NaN,NaN,NaN,Alfani,SHOE,"Women's Shoes, Women's",NaN,NaN,...,NaN,NaN,NaN,"Accessories, Pumps",NaN,NaN,NaN,Peep Toe,NaN,NaN


# Merge ImageData with AttributeData

In [12]:
merged_df = raw_image.merge(cat_df_pivoted, left_on="PRODUCT_ID", right_on="PROD_ID")
print("\n", "shape merged_df", merged_df.shape)
merged_df.tail()


 shape merged_df (5628, 38)


,PRODUCT_ID,IMAGE_ID,IMAGE_FILE,Age Group,Age Range (Toys),Apparel Occasion,Boot Height,Boot Type,Brand,Category,...,Sandal Type,Shoe Style,Shoe Trend,Shoe Type,Size Range,Special Size,Sub Brand,Toe Type,Trends,Weather
5623,6009291,9058257,6/0/6009291_9058257.jpg,Adult,NaN,NaN,NaN,NaN,Call It Spring,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,NaN,"Pointy, Closed",NaN,NaN
5624,5346211,9037824,5/3/5346211_9037824.jpg,Adult,NaN,NaN,NaN,NaN,Nine West,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,NaN,Round,NaN,NaN
5625,2999896,3887016,2/9/2999896_3887016.jpg,Adult,NaN,NaN,NaN,NaN,Nine West,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,NaN,Pointy,NaN,NaN
5626,5918122,9352623,5/9/5918122_9352623.jpg,Adult,NaN,NaN,NaN,NaN,Blue by Betsey Johnson,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,Blue by Betsey Johnson,Peep Toe,NaN,NaN
5627,5612366,9650387,5/6/5612366_9650387.jpg,Adult,NaN,NaN,NaN,NaN,Michael Kors,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,MICHAEL Michael Kors,Pointy,NaN,NaN


# Visualize final df

In [13]:
columns_names = ['IMAGE_FILE', 'Category', 'Shoe Type']
print(SELECTED_ATTRIBUTES)
columns_names.extend(SELECTED_ATTRIBUTES)
print(columns_names)
viz_df = merged_df[columns_names]
print("\n", "shape viz_df", viz_df.shape)
print("\n", viz_df['Shoe Type'].unique())
viz_df.head()

[]
['IMAGE_FILE', 'Category', 'Shoe Type']

 shape viz_df (5628, 3)

 ['Pumps' 'Pumps, Wedges' 'Pumps, Sandals' 'Pumps, Wedges, Sandals'
 'Boots, Pumps, Booties' 'Pumps, Mules' 'Pumps, Flats']


,IMAGE_FILE,Category,Shoe Type
0,8/0/8062133_8968094.jpg,SHOE,Pumps
1,8/0/8062133_8968082.jpg,SHOE,Pumps
2,8/0/8062133_8968100.jpg,SHOE,Pumps
3,8/0/8062133_8968088.jpg,SHOE,Pumps
4,8/0/8062133_9939242.jpg,SHOE,Pumps


# Save df

In [14]:
merged_df.to_hdf(SAVE_OUTPUT_AS, key="macys_images")
print("Saved")

Saved


/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['IMAGE_FILE', 'Age Group', 'Age Range (Toys)', 'Apparel Occasion', 'Boot Height', 'Boot Type', 'Brand', 'Category', 'Category of Business', 'Cold Weather & Rain', 'Color Family', 'Comfort Technology', 'Department Type', 'Evening & Bridal Shoes', 'Fabric', 'Flat Type', 'Gender', 'Heel', 'Impulse Trends', 'More Colors Available', 'NRF_ID', 'Occasion', 'Overlay Bottom', 'Product Color', 'Pump Type', 'RTW Trends', 'Sandal Type', 'Shoe Style', 'Shoe Trend', 'Shoe Type', 'Size Range', 'Special Size', 'Sub Brand', 'Toe Type', 'Trends', 'Weather']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


# Roload for QA

In [15]:
reloaded_df = pd.read_hdf(SAVE_OUTPUT_AS)
print("\n", "shape reloaded", reloaded_df.shape)
reloaded_df.head()


 shape reloaded (5628, 38)


,PRODUCT_ID,IMAGE_ID,IMAGE_FILE,Age Group,Age Range (Toys),Apparel Occasion,Boot Height,Boot Type,Brand,Category,...,Sandal Type,Shoe Style,Shoe Trend,Shoe Type,Size Range,Special Size,Sub Brand,Toe Type,Trends,Weather
0,8062133,8968094,8/0/8062133_8968094.jpg,Adult,NaN,NaN,NaN,NaN,Franco Sarto,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,NaN,Pointy,NaN,NaN
1,8062133,8968082,8/0/8062133_8968082.jpg,Adult,NaN,NaN,NaN,NaN,Franco Sarto,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,NaN,Pointy,NaN,NaN
2,8062133,8968100,8/0/8062133_8968100.jpg,Adult,NaN,NaN,NaN,NaN,Franco Sarto,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,NaN,Pointy,NaN,NaN
3,8062133,8968088,8/0/8062133_8968088.jpg,Adult,NaN,NaN,NaN,NaN,Franco Sarto,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,NaN,Pointy,NaN,NaN
4,8062133,9939242,8/0/8062133_9939242.jpg,Adult,NaN,NaN,NaN,NaN,Franco Sarto,SHOE,...,NaN,NaN,NaN,Pumps,NaN,NaN,NaN,Pointy,NaN,NaN


In [16]:
for cat in SELECTED_CATEGORIES:
    tmp_df = reloaded_df[reloaded_df['Category'] == "SHOE"]
    print(cat, len(tmp_df))

PUMP 5628
PUMPS 5628
